# Users

This notebook contains examples of using the ArcGIS API for Python to manage users in an ArcGIS Online portal. 

Each cell is largely independent of every other cell, with the exception that most assume the existence of a portal connection stored in a variable named `gis`. The creation of this object is covered in the first section of the notebook.

Usernames used throughout this notebook are specific to my organization. You will need to change them to something appropriate for your organization.

All API references in this notebook are from the [arcgis.gis](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html) module: [admin.CreditManager](https://developers.arcgis.com/python/api-reference/arcgis.gis.admin.html#creditmanager), [ContentManager](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#contentmanager), [GIS](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#gis), [User](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#user), and [UserManager](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#usermanager)



## Table of Contents

* [Connect to the portal](#portal-connection)
    * [Connect through ArcGIS Pro](#pro-connection)
    * [Interactively provide password](#provide-password)
    * [Store credentials locally](#store-credentials)
    * [Other connection methods](#other-connection)
* [Create `User` objects](#create-user-objects)
    * [List all users in an organization](#list-users)
    * [Search for users](#search-users)
    * [Get a specific user](#get-user)
* [Get information about a user](#get-info)
    * [Display user information](#display-info)
    * [Access user properties](#access-properties)
    * [Find items owned by a user](#find-items)
* [Manage users](#manage)
    * [Create multiple users](#create)
    * [Update a user's information](#update)
    * [Reset a user's password](#reset)
    * [Update a user's license type](#update-license)
    * [Update a user's role](#update-role)
    * [Allocate credits to a user](#allocate)
    * [Reassign a user's content](#reassign)
    * [Delete a single user](#delete-single)
    * [Delete multiple users](#delete-multiple)




## Connect to the portal <a id='portal-connection'></a>


In order to manage users, you will need to connect to ArcGIS Online using the [`GIS` class in the `arcgis.gis` module](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#gis). You could do that insecurely by putting your username and password into your code. This is a bad idea, don't do that. Instead, use one of the methods below.

### Connect through ArcGIS Pro <a id='pro-connection'></a>


If you are running this notebook in ArcGIS Pro, and are logged in to your portal through ArcGIS Pro, the following code will connect to your portal using the credentials you already provided to ArcGIS Pro.

In [ ]:
from arcgis.gis import GIS
gis = GIS("pro")

### Interactively provide password <a id='provide-password'></a>

If you are working interactively with the notebook, the following code will prompt you to type your password.

In [4]:
from arcgis.gis import GIS
gis = GIS(url="https://arcgis.com", username="t.ormsby")

### Store credentials locally <a id='store-credentials'></a>


You can keep your credentials on your local machine and reference them from your code, instead of providing the credentials directly. To store you credentials, you will first need to create a profile, which does require you to provide your username and password.

Make sure you do not save the code you used to generate the profile.

In [7]:
from arcgis.gis import GIS
gis = GIS(
    url="https://arcgis.com", 
    username="t.ormsby", 
    password="never_store_real_passwords_in_code", 
    profile="ormsby_agol"
)

Profile generation is a one-time event. Once you have generated the profile, you can use it to connect to your portal. Because the profile is stored locally, you will not be able to use this code to connect to the portal from a different machine.

In [15]:
from arcgis.gis import GIS
gis = GIS(profile="ormsby_agol")

### Other connection methods <a id='other-connection'></a>


There are other methods than the three above, but they are harder to use. If you need the capabilities provided by access tokens, PKI, LDAP, or OAuth, you can read more about it in the [Esri documentation for working with different authentication schemes](https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/).

## Create `User` objects <a id='create-user-objects'></a>

In order to manage users, you will need to represent them as some data structure that Python can manipulate. The [`UserManager` class](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#usermanager) can create these `User` objects. An instance of this class is available as the `users` property of a `GIS` object

### List all users in an organization <a id='list-users'></a>

The [`org_search` method of the UserManager object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.UserManager.org_search) will generate a tuple of `User` objects. By default, it will return all users in your organization. <a id='org_search'></a>

In [224]:
all_users = gis.users.org_search()
for user in all_users:
    print(user)

<User username:ormsby_guest1>
<User username:t.ormsby>
<User username:ormsby_guest2>


### Search for users <a id='search-users'></a>

You can also use `org_search` to find users in your organization that match a query and sort the `User` objects returned.

In [225]:
some_users = gis.users.org_search(query="ormsby_guest", sort_field="username", sort_order="desc")
for user in some_users:
    print(user)

<User username:ormsby_guest2>
<User username:ormsby_guest1>


In addition to `org_search`, you can use the [`search`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.UserManager.search) and [`advanced_search`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.UserManager.advanced_search) methods to search for users. These methods can search across ArcGIS Online.<a id='search'></a>

### Get a specific user <a id='get-user'></a>

The [`get` method of the UserManager object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.UserManager.get)  will return a single `User` object that matches the provided username. The `get` method is not limited to a specific organization and can return any ArcGIS Online user. <a id='get'></a>

In [226]:
one_user = gis.users.get(username="ormsby_guest1")
print(one_user)

<User username:ormsby_guest1>


## Get information about a user <a id='get-info'></a>

Once you have a `User` object, you can access information about that user. 

### Display user information <a id='display-info'></a>

The code below uses the Jupyter Notebook `display` function to show a card for a user, including a link to their profile page.

In [227]:
user = gis.users.get("ormsby_guest1")
display(user)

<User username:ormsby_guest1>

### Access user properties <a id='access-properties'></a>

You can also access specific properties of a `User` object. The code below prints some example information you can access. The [API Reference for the `User` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#user) has the complete list. <a id='properties'></a>

In [228]:
user = gis.users.get("ormsby_guest1")
print(user.fullName)
print(user.availableCredits)
print(user.role)
print(user.groups)

Ormsby Guest1
10.0
org_publisher
[]


Most objects in Python have a `__dict__` property that shows all the object's properties. The code below leverages the `__dict__` property to represent the `User` object as a Python dictionary.

In [229]:
user = gis.users.get("ormsby_guest1")
user.__dict__

{'_gis': GIS @ https://ormsby.maps.arcgis.com version:2023.2,
 '_portal': <arcgis.gis._impl._portalpy.Portal at 0x7fda24becd90>,
 '_user_id': 'ormsby_guest1',
 'thumbnail': None,
 '_workdir': '/tmp',
 '_invitemgr': None,
 '_hydrated': False,
 'username': 'ormsby_guest1',
 'udn': None,
 'id': '6626b8616d794725978b22fafdbdcc44',
 'fullName': 'Ormsby Guest1',
 'availableCredits': 10.0,
 'assignedCredits': 10.0,
 'categories': [],
 'emailStatus': 'notverified',
 'firstName': 'Ormsby',
 'lastName': 'Guest1',
 'preferredView': None,
 'description': None,
 'email': 't.ormsby@esri.com',
 'userType': 'arcgisonly',
 'idpUsername': None,
 'favGroupId': 'cfb0adf777ca48a383ae610543ab9356',
 'lastLogin': -1,
 'mfaEnabled': False,
 'validateUserProfile': True,
 'storageUsage': 445780599,
 'storageQuota': 2199023255552,
 'orgId': 'jgEF2qh8RkmonUq2',
 'role': 'org_publisher',
 'privileges': ['features:user:edit',
  'portal:publisher:bulkPublishFromDataStores',
  'portal:publisher:createDataPipelines',


### Find items owned by a user <a id='find-items'></a>


<a id='item_search'></a> There is an [`items` method of a `User` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.User.items) but it can only be used to check items in a specified folder. 

For that reason, the code below uses the [`search` method of the `ContentManager` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.ContentManager.search) to generate a list of `Item` objects owned by a given username.

In [230]:
gis.content.search(query="owner:ormsby_guest1", max_items=20)

[<Item title:"ExampleMap3" type:Web Map owner:ormsby_guest1>,
 <Item title:"ExampleMap1" type:Web Map owner:ormsby_guest1>,
 <Item title:"ExampleMap2" type:Web Map owner:ormsby_guest1>]

## Manage users <a id='manage'></a>

With a `User` object, you can programatically manage a user. The following examples give a good idea of the types of things you can do with users, but are not comprehensive. The API Reference for the [User](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#user) and [UserManager](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#user) classes provides additional details.

### Create multiple users <a id='create'></a>

<a id='create'></a>The code below creates three new Viewer accounts in the built-in identity store by using the [`create` method of the `UserManager` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.UserManager.create). This operation creates only one new user at a time, but multiple users can be created by using a loop. Specifying a password of `None` requires the user to set their password when they first attempt to log in to the portal.

Note that ArcGIS Online usernames that use the built-in identity store must be globally unique across all ArcGIS Online users. 

In [242]:
user_data = [
    {"uname": "fabb44dc", "fname": "Test", "lname": "User1", "email": "testuser1@example.com"},
    {"uname": "cedd2845", "fname": "Test", "lname": "User2", "email": "testuser2@example.com"},
    {"uname": "e23fb959", "fname": "Test", "lname": "User3", "email": "testuser3@example.com"}
]

for user in user_data:
    new_user = gis.users.create(
        username=user["uname"],
        password=None,
        firstname=user["fname"],
        lastname=user["lname"],
        email=user["email"],
        role="iAAAAAAAAAAAAAAA", # Viewer role id
        user_type="viewerUT",
        credits=0
    )
    display(new_user)

<User username:fabb44dc>

<User username:cedd2845>

<User username:e23fb959>

### Update a user's information <a id='update'></a>

The code below updates a user's name, email, and description, using the [`update` method of the `User` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.User.update). The documentation has a comprehensive list of properties that can be changed with this method. <a id='update'></a>

In [233]:
user = gis.users.get("fabb44dc")
user.update(
    first_name="New First Name",
    last_name="New Last Name",
    email="newname@example.com",
    description="I feel like a brand new person"   
)
display(user)


<User username:fabb44dc>

### Reset a user's password <a id='reset'></a>

Accounts that use IT-managed identity stores like Active Directory or SAML cannot be managed with the ArcGIS API for Python. But an account that uses the built-in identity store can be. The code below uses the [`reset` method of a `User` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.User.reset) to send a user an email with a temporary password and instructions to reset it the next time they log in.

In [234]:
user = gis.users.get("fabb44dc")
user.reset(password=None, reset_by_email=True)

True

### Update a user's license type <a id='update-license'></a>

A user's license type governs what types of roles and privileges they can be granted. In this example, the user has a Viewer license type. The code below changes that to a Creator license type using the [`update_license_type` method of a `User` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.User.update_license_type).

In [235]:
user = gis.users.get("fabb44dc")
user.update_license_type("CreatorUT")

True

### Update a user's role <a id='update-role'></a>

A user's role governs the privileges they have in the portal, and must be compatible with their license type. In this example, the user has a Viewer role. The code below changes that to a Publisher role using the [`update_role` method of the `User` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.User.update_role)

In [236]:
user = gis.users.get("fabb44dc")
user.update_role("org_publisher")

True

### Allocate credits to a user <a id='allocate'></a>

The code below uses the [`allocate` method of the `CreditManager` class in the admin submodule](https://developers.arcgis.com/python/api-reference/arcgis.gis.admin.html#arcgis.gis.admin.CreditManager.allocate) to allocate credits to a user based on their username.

In [237]:
gis.admin.credits.allocate(username="fabb44dc", credits=10)

True

### Reassign a user's content <a id='reassign'></a>

The code below reassigns all of a user's items and groups to a different user with the [`reassign_to` method of a `User` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.User.reassign_to). This operation can only be performed by an administrator, occurs immediately, and cannot be undone.

This operation recreates the original folder structure of items in the new owner's content.

In [238]:
user1 = gis.users.get("ormsby_guest1")
user2 = gis.users.get("fabb44dc")
user1.reassign_to(target_username=user2)

True

The code below uses the [`transfer_content` method of a `User` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.User.transfer_content) to transfer the content of a folder to another user. This asyncronous operation may take up to 15 minutes to complete. <a id='transfer_content'></a>

This operation transfers all items to a single folder in the new owner's content, regardless of the original folder structure.

In [239]:
user1 = gis.users.get("ormsby_guest1")
user2 = gis.users.get("fabb44dc")
user2.transfer_content(target_user=user1)

<Future at 0x7fda1ce16590 state=finished returned dict>

### Delete a single user <a id='delete-single'></a>

The code below uses the [`delete` method of a `User` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.User.delete) to delete that user. Users that own content or groups will need to have them reassigned to another user.

In [243]:
user = gis.users.get("fabb44dc")
user.delete()

True

### Delete multiple users <a id='delete-multiple'></a>

There is a [`delete_users` method of the `UserManager` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.UserManager.delete_users) that accepts a list of `User` objects. This operation, however, does not actually delete the users, it merely removes them from the organization. You cannot see them when searching for users, but you can still access their `User` object with the `get` method, and you cannot create new users with the same username as these removed users.  

For that reason, the code below deletes multiple users by looping through users and using the `delete` method. This strategy is slower than the `delete_users` method, but more effective in permanently deleting users.

In [244]:
unames = ["cedd2845","e23fb959"]
for uname in unames:
    user = gis.users.get(uname)
    user.delete()